In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import cftime
import datetime
from datetime import date
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
from matplotlib.cm import ScalarMappable
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
import numpy
import pandas
from PIL import Image
import random
import xarray as xr

In [2]:
Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Data/Output_Files_V6/'

In [3]:
# Open File
def Open_File(File):
    DF = pandas.read_csv(File)
    DF = DF.drop("Unnamed: 0", axis=1)
    return (DF)

In [4]:
# Open Each File
def Files_Open(Model, Diri):
    Data_DF = Open_File(Diri+Model+'_Data_SubsetB_Output_V6.csv')
    ET_DF = Open_File(Diri+Model+'_ET_SubsetB_Output_V6.csv')
    Codes_DF = Open_File(Diri+Model+'_Codes_Output_V6.csv')
# Edit Time Format
    Time, Begin_Time, Compl_Time, Trop_Peak_Time, Peak_Time, Birth_Time, Death_Time = [], [], [], [], [], [], []
    for i in range(len(Data_DF)):
        Time.append(Datetime(Data_DF["Time(Z)"][i]))
    for j in range(len(ET_DF)):
        Begin_Time.append(Datetime(ET_DF["ET Begin Time"][j]))
        Compl_Time.append(Datetime(ET_DF["ET Complete Time"][j]))
        Trop_Peak_Time.append(Datetime(ET_DF["Trop Peak Time"][j]))
        Peak_Time.append(Datetime(ET_DF["Peak Time"][j]))
        Birth_Time.append(Datetime(ET_DF["Genesis Time"][j]))
        Death_Time.append(Datetime(ET_DF["Dissipation Time"][j]))
    Data_DF["Time(Z)"] = Time
    ET_DF["ET Begin Time"] = Begin_Time
    ET_DF["ET Complete Time"] = Compl_Time
    ET_DF["Trop Peak Time"] = Trop_Peak_Time
    ET_DF["Peak Time"] = Peak_Time
    ET_DF["Genesis Time"] = Birth_Time
    ET_DF["Dissipation Time"] = Death_Time
    return (Data_DF, ET_DF, Codes_DF)

In [5]:
def Datetime(Time):
    try:
        New_Time = datetime.datetime.strptime(Time, '%Y-%m-%d %H:%M:%S')
    except:
        New_Time = numpy.nan
    return (New_Time)

In [6]:
# Create Phase Distribution Bins
def Create_Bins(Min, Max, Bin_Width):
    Bins = numpy.arange(Min, Max+Bin_Width, Bin_Width)
    return (Bins)
Lat_Bins = Create_Bins(10,60,5)
Lon_Bins = Create_Bins(-100,20,5)

In [7]:
Num_Years = numpy.array([90,93,93])

In [8]:
# Function to Find Distance Between Two Points
def Find_Distance(y1, y2, x1, x2):
    Start_Lat = y1 * numpy.pi / 180
    End_Lat = y2 * numpy.pi / 180
    Start_Lon = x1 * numpy.pi / 180
    End_Lon = x2 * numpy.pi / 180
    Lat_Diff = End_Lat - Start_Lat
    Lon_Diff = End_Lon - Start_Lon
    Earth_Rad = 6378
    Distance = 2 * Earth_Rad * numpy.sqrt((numpy.sin(Lat_Diff/2))**2 + \
    numpy.cos(Start_Lat) * numpy.cos(End_Lat) * (numpy.sin(Lon_Diff/2))**2)
    return (Distance)

In [9]:
# Find a Specific Storm Within the DataFrame
def Find_Storm(DF, Code):
    DF_Storm = DF[DF["Code"] == Code].reset_index()
    return (DF_Storm)

In [10]:
Control_Data, Control_ET, Control_Codes = Files_Open("Control", Diri)
RCP45_Data, RCP45_ET, RCP45_Codes = Files_Open("RCP45", Diri)
RCP85_Data, RCP85_ET, RCP85_Codes = Files_Open("RCP85", Diri)

In [67]:
# Create List of Monthly File Names
def Monthly_File_Names(Model, Years, Months, File_Names):
    for j in range(12):
        for i in range(len(Years)):
            File_Name = Model + '/' + Model + '.cam.h0.' + str(Years[i]) + '-' + str(Months[j]) + '.nc_regrid.nc'
            File_Names[j].append(File_Name)
    return (File_Names)

In [210]:
Control_Years = Create_Bins(1985,2014,1)
RCP45_Years = Create_Bins(2069,2099,1)
RCP85_Years = Create_Bins(2069,2099,1)
Months = ['01','02','03','04','05','06','07','08','09','10','11','12']
Months_Name = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [118]:
# Define File Names For Control
Control_Monthly_Files = []
for j in range(12):
    Control_Monthly_Files.append([])
Control_Monthly_Files = Monthly_File_Names('CHEY.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.002', Control_Years, Months, Control_Monthly_Files)
Control_Monthly_Files = Monthly_File_Names('CHEY.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.004', Control_Years, Months, Control_Monthly_Files)
Control_Monthly_Files = Monthly_File_Names('CHEY.VR28.NATL.REF.CAM5.4CLM5.0.dtime900', Control_Years, Months, Control_Monthly_Files)

In [119]:
# Define File Names For RCP4.5
RCP45_Monthly_Files = []
for j in range(12):
    RCP45_Monthly_Files.append([])
RCP45_Monthly_Files = Monthly_File_Names('CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900', RCP45_Years, Months, RCP45_Monthly_Files)
RCP45_Monthly_Files = Monthly_File_Names('CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.002', RCP45_Years, Months, RCP45_Monthly_Files)
RCP45_Monthly_Files = Monthly_File_Names('CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003', RCP45_Years, Months, RCP45_Monthly_Files)

In [120]:
# Define File Names For RCP8.5
RCP85_Monthly_Files = []
for j in range(12):
    RCP85_Monthly_Files.append([])
RCP85_Monthly_Files = Monthly_File_Names('CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900', RCP85_Years, Months, RCP85_Monthly_Files)
RCP85_Monthly_Files = Monthly_File_Names('CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003', RCP85_Years, Months, RCP85_Monthly_Files)
RCP85_Monthly_Files = Monthly_File_Names('CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.004', RCP85_Years, Months, RCP85_Monthly_Files)

In [131]:
# Create Function to Open Monthly Files
def Open_Monthly_File(File_Names):
    Diri = '/glade/campaign/univ/upsu0032/Hyperion_ET/'
    for i in range(len(File_Names)):
#    for i in range(12):
        Month_DS = xr.open_dataset(Diri + File_Names[i])
        Month_DS_New = Month_DS.where((Month_DS.lon >= -100) & (Month_DS.lon <= 0) & \
        (Month_DS.lat >= 0) & (Month_DS.lat <= 60) & (Month_DS.plev >= 25000), drop=True)
# Edit Time Stamp To Make Sure Dates Don't Overlap
        if i < (len(File_Names) / 3):
            Month_DS_New['time'] = Month_DS_New['time'] - numpy.timedelta64(16,'D')
        elif i < (len(File_Names) / 3 * 2):
            Month_DS_New['time'] = Month_DS_New['time'] - numpy.timedelta64(10,'D')
        else:
            Month_DS_New['time'] = Month_DS_New['time'] - numpy.timedelta64(4,'D')
# Combine Datasets
        if i == 0:
            Month_DS_Combine = Month_DS_New
        else:
            Month_DS_Combine = xr.concat([Month_DS_Combine, Month_DS_New], dim='time')
        if i % 10 == 0:
            print (i)
    return (Month_DS_Combine)

In [132]:
Control_DS_Jan = Open_Monthly_File(Control_Monthly_Files[0])
Control_DS_Feb = Open_Monthly_File(Control_Monthly_Files[1])
Control_DS_Mar = Open_Monthly_File(Control_Monthly_Files[2])

In [ ]:
Control_DS_Apr = Open_Monthly_File(Control_Monthly_Files[3])
Control_DS_May = Open_Monthly_File(Control_Monthly_Files[4])
Control_DS_Jun = Open_Monthly_File(Control_Monthly_Files[5])

In [125]:
Control_DS_Jul = Open_Monthly_File(Control_Monthly_Files[6])
Control_DS_Aug = Open_Monthly_File(Control_Monthly_Files[7])
Control_DS_Sep = Open_Monthly_File(Control_Monthly_Files[8])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76

In [133]:
Control_DS_Oct = Open_Monthly_File(Control_Monthly_Files[9])
Control_DS_Nov = Open_Monthly_File(Control_Monthly_Files[10])
Control_DS_Dec = Open_Monthly_File(Control_Monthly_Files[11])

In [134]:
RCP45_DS_Jan = Open_Monthly_File(RCP45_Monthly_Files[0])
RCP45_DS_Feb = Open_Monthly_File(RCP45_Monthly_Files[1])
RCP45_DS_Mar = Open_Monthly_File(RCP45_Monthly_Files[2])

In [ ]:
RCP45_DS_Apr = Open_Monthly_File(RCP45_Monthly_Files[3])
RCP45_DS_May = Open_Monthly_File(RCP45_Monthly_Files[4])
RCP45_DS_Jun = Open_Monthly_File(RCP45_Monthly_Files[5])

In [127]:
RCP45_DS_Jul = Open_Monthly_File(RCP45_Monthly_Files[6])
RCP45_DS_Aug = Open_Monthly_File(RCP45_Monthly_Files[7])
RCP45_DS_Sep = Open_Monthly_File(RCP45_Monthly_Files[8])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67

In [141]:
RCP45_DS_Oct = Open_Monthly_File(RCP45_Monthly_Files[9])
RCP45_DS_Nov = Open_Monthly_File(RCP45_Monthly_Files[10])
RCP45_DS_Dec = Open_Monthly_File(RCP45_Monthly_Files[11])


KeyboardInterrupt



In [136]:
RCP85_DS_Jan = Open_Monthly_File(RCP85_Monthly_Files[0])
RCP85_DS_Feb = Open_Monthly_File(RCP85_Monthly_Files[1])
RCP85_DS_Mar = Open_Monthly_File(RCP85_Monthly_Files[2])

In [ ]:
RCP85_DS_Apr = Open_Monthly_File(RCP85_Monthly_Files[3])
RCP85_DS_May = Open_Monthly_File(RCP85_Monthly_Files[4])
RCP85_DS_Jun = Open_Monthly_File(RCP85_Monthly_Files[5])

In [128]:
RCP85_DS_Jul = Open_Monthly_File(RCP85_Monthly_Files[6])
RCP85_DS_Aug = Open_Monthly_File(RCP85_Monthly_Files[7])
RCP85_DS_Sep = Open_Monthly_File(RCP85_Monthly_Files[8])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67

In [137]:
RCP85_DS_Oct = Open_Monthly_File(RCP85_Monthly_Files[9])
RCP85_DS_Nov = Open_Monthly_File(RCP85_Monthly_Files[10])
RCP85_DS_Dec = Open_Monthly_File(RCP85_Monthly_Files[11])

In [129]:
RCP85_DS_Jul

<xarray.Dataset>
Dimensions:    (time: 93, lat: 61, lon: 101, plev: 9, nbnd: 2)
Coordinates:
  * lat        (lat) float64 0.0 1.0 2.0 3.0 4.0 ... 56.0 57.0 58.0 59.0 60.0
  * lon        (lon) float64 -100.0 -99.0 -98.0 -97.0 ... -3.0 -2.0 -1.0 0.0
  * plev       (plev) float64 1e+05 9.25e+04 8.5e+04 ... 4e+04 3e+04 2.5e+04
  * time       (time) datetime64[ns] 2069-07-16 2070-07-16 ... 2099-07-28
Dimensions without coordinates: nbnd
Data variables: (12/74)
    CLDHGH     (time, lat, lon, plev) float32 0.5367 0.5367 ... 0.3466 0.3466
    CLDICE     (time, plev, lat, lon) float32 0.0 9.088e-33 ... 1.492e-06
    CLDLIQ     (time, plev, lat, lon) float32 5.84e-08 8.533e-08 ... 2.246e-13
    CLDLOW     (time, lat, lon, plev) float32 0.3522 0.3522 ... 0.8032 0.8032
    CLDMED     (time, lat, lon, plev) float32 0.03862 0.03862 ... 0.3402 0.3402
    CLDTOT     (time, lat, lon, plev) float32 0.6699 0.6699 ... 0.8811 0.8811
    ...         ...
    Z3         (time, plev, lat, lon) float32 120.9 120.9 ... 1.071e+04
    area       (time, lat, lon, plev) float64 0.0003046 0.0003046 ... 0.0001523
    gw         (time, lat, lon, plev) float64 0.01745 0.01745 ... 0.008727
    lat_bnds   (time, lat, nbnd, lon, plev) float64 -0.5 -0.5 -0.5 ... 60.5 60.5
    lon_bnds   (time, lon, nbnd, lat, plev) float64 -100.5 -100.5 ... 0.5 0.5
    time_bnds  (time, nbnd, lon, lat, plev) datetime64[ns] 2069-07-01 ... 209...
Attributes: (12/21)
    ne:                         0
    np:                         4
    Conventions:                CF-1.0
    source:                     CAM
    case:                       b40_1850_2d_r07c5cn_160jp
    title:                      Regridded version of tmp.nc
    ...                         ...
    history_of_appended_files:  Sat Apr 12 18:59:06 2025: Appended file /glad...
    remap_script:               ncremap
    remap_hostname:             casper-login2
    remap_version:              5.3.1
    map_file:                   /glade/work/zarzycki/maps/hyperion/map_ne0np4...
    input_file:                 /glade/u/home/zarzycki/scratch/hyperion/CHEY....

In [138]:
# Create Dictionaries to Store Datasets
Control_DS_Dict = {"Jan": Control_DS_Jan, "Feb": Control_DS_Feb, "Mar": Control_DS_Mar, \
"Apr": Control_DS_Apr, "May": Control_DS_May, "Jun": Control_DS_Jun, \
"Jul": Control_DS_Jul, "Aug": Control_DS_Aug, "Sep": Control_DS_Sep, \
"Oct": Control_DS_Oct, "Nov": Control_DS_Nov, "Dec": Control_DS_Dec}
RCP45_DS_Dict = {"Jan": RCP45_DS_Jan, "Feb": RCP45_DS_Feb, "Mar": RCP45_DS_Mar, \
"Apr": RCP45_DS_Apr, "May": RCP45_DS_May, "Jun": RCP45_DS_Jun, \
"Jul": RCP45_DS_Jul, "Aug": RCP45_DS_Aug, "Sep": RCP45_DS_Sep, \
"Oct": RCP45_DS_Oct, "Nov": RCP45_DS_Nov, "Dec": RCP45_DS_Dec}
RCP85_DS_Dict = {"Jan": RCP85_DS_Jan, "Feb": RCP85_DS_Feb, "Mar": RCP85_DS_Mar, \
"Apr": RCP85_DS_Apr, "May": RCP85_DS_May, "Jun": RCP85_DS_Jun, \
"Jul": RCP85_DS_Jul, "Aug": RCP85_DS_Aug, "Sep": RCP85_DS_Sep, \
"Oct": RCP85_DS_Oct, "Nov": RCP85_DS_Nov, "Dec": RCP85_DS_Dec}

In [207]:
Output_Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Data/NetCDF_Files_V6/'

In [208]:
# Output DS to netCDF File
def Output_DS(DS, Model, Month):
    File_Name = str(Model+'_Output_'+Month+'_V6.nc')
    DS.to_netcdf(Output_Diri+File_Name)

In [211]:
# Output Control Files For Each Month
for i in range(12):
    DS = Control_DS_Dict[Months_Name[i]]
    Output_DS(DS, "Control", Months_Name[i])

In [ ]:
# Output RCP4.5 Files For Each Month
for i in range(12):
    DS = RCP45_DS_Dict[Months_Name[i]]
    Output_DS(DS, "RCP45", Months_Name[i])

In [ ]:
# Output RCP8.5 Files For Each Month
for i in range(12):
    DS = RCP85_DS_Dict[Months_Name[i]]
    Output_DS(DS, "RCP85", Months_Name[i])